In [ ]:
import gradio as gr
from obj_detect import process_video
from zoom import zoom_save_gif
from demo_hmr import human_pose_estimation, integration, rearrange_data

/home/ziyan/anaconda3/envs/4D-humans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to process the input list
def process_list(input_text):
    string_list = [item.strip() for item in input_text.split(",") if item.strip()]
    return string_list

In [ ]:
def gradio_process_video(video_file, detector, 
                         box_threshold_1, box_threshold_2, 
                         text_threshold_1, text_threshold_2, 
                         confidence_1, confidence_2, 
                         iou_1, iou_2, 
                         frame_len, frame_stride, gif_duration,
                         use_prior_data, use_pose, out_file, fps, 
                         selected_ids, selected_labels, nearest_to_ball,
                         x_padding, y_padding, x_interp, y_interp, zoom_interp):
    box_threshold = [box_threshold_1, box_threshold_2]
    text_threshold = [text_threshold_1, text_threshold_2]
    yolo_confidence = [confidence_1, confidence_2]
    yolo_iou_threhold = [iou_1, iou_2]

    if not use_prior_data:
        process_video(video_path=video_file,
                    detector=detector,
                    dino_box_threshold=box_threshold,
                    dino_text_threshold=text_threshold,
                    yolo_confidence=yolo_confidence,
                    yolo_iou_threshold=yolo_iou_threhold,
                    frame_len=frame_len,
                    frame_stride=frame_stride,
                    gif_duration=gif_duration)
        
    if use_pose and not use_prior_data:
        human_pose_estimation(img_folder="output_video_frames", out_folder="output")
        rearrange_data(output_folder="output")

    selected_ids = process_list(selected_ids)
    selected_labels = process_list(selected_labels)
    
    return zoom_save_gif(out_file, use_pose, fps, selected_ids, selected_labels, nearest_to_ball,
                         x_padding, y_padding, x_interp, y_interp, zoom_interp)

In [4]:
def update_inputs(detector):
    if detector == "Grounding DINO":
        # Show DINO sliders, hide YOLO sliders
        return [gr.update(visible=True), gr.update(visible=False)]
    else:
        # Show YOLO sliders, hide DINO sliders
        return [gr.update(visible=False), gr.update(visible=True)]

In [ ]:
with gr.Blocks(title="Autozoom Basketball Game") as demo:
    with gr.Row():
        with gr.Column(scale=1):  # Left half for inputs
            video_file = gr.Video(label="Upload Video")
            detector = gr.Radio(choices=["Grounding DINO", "YOLO"], label="Choose a detector")

            with gr.Group(visible=False) as dino_sliders:
                box_threshold_1 = gr.Slider(0, 1, value=0.35, step=0.05, label="Box Threshold (DINO): Player/Referee")
                box_threshold_2 = gr.Slider(0, 1, value=0.35, step=0.05, label="Box Threshold (DINO): Basketball")
                text_threshold_1 = gr.Slider(0, 1, value=0.35, step=0.05, label="Text Threshold (DINO): Player/Referee")
                text_threshold_2 = gr.Slider(0, 1, value=0.35, step=0.05, label="Text Threshold (DINO): Basketball")

            with gr.Group(visible=False) as yolo_sliders:
                confidence_1 = gr.Slider(0, 1, value=0.3, step=0.05, label="Confidence (YOLO): Player/Referee")
                confidence_2 = gr.Slider(0, 1, value=0.75, step=0.05, label="Confidence (YOLO): Basketball")
                iou_1 = gr.Slider(0, 1, value=0.7, step=0.05, label="IOU Threshold (YOLO): Player/Referee")
                iou_2 = gr.Slider(0, 1, value=0.5, step=0.05, label="IOU Threshold (YOLO): Basketball")

            frame_len = gr.Slider(1, 500, value=100, step=1, label="Frame Length")
            frame_stride = gr.Slider(1, 50, value=3, step=1, label="Frame Stride")
            gif_duration = gr.Slider(1, 500, value=100, step=1, label="GIF Duration")
            use_prior_data = gr.Checkbox(label="Use Previous Data")
            use_pose = gr.Checkbox(label="Use Pose Data")
            out_file = gr.Textbox(value="zoomed.gif", placeholder="Out Filename")
            fps = gr.Slider(1, 60, value=12, step=1, label="FPS")
            selected_ids = gr.Textbox(placeholder="IDs Targeted (seperated by commas)")
            selected_labels = gr.Textbox(value="basketball", placeholder="Labels Targeted (seperated by commas)")
            nearest_to_ball = gr.Slider(0, 10, value=3, step=1, label="Closest to Ball")
            x_padding = gr.Slider(1, 100, value=20, step=1, label="X Padding")
            y_padding = gr.Slider(1, 100, value=20, step=1, label="Y Padding")
            x_interp = gr.Slider(0, 1, value=0.5, step=0.05, label="X Interpolation")
            y_interp = gr.Slider(0, 1, value=0.5, step=0.05, label="Y Interpolation")
            zoom_interp = gr.Slider(0, 1, value=0.5, step=0.05, label="Zoom Interpolation")

            submit = gr.Button("Process")
        
        with gr.Column(scale=1):  # Right half for outputs
            output = gr.Image(type="filepath", label="Output")

    # Adjust outputs: first dino_sliders group, then yolo_sliders group
    detector.change(
        update_inputs,
        inputs=[detector],
        outputs=[dino_sliders, yolo_sliders]
    )

    # Submit button action
    submit.click(
        gradio_process_video,
        inputs=[video_file, detector, 
                box_threshold_1, box_threshold_2, 
                text_threshold_1, text_threshold_2, 
                confidence_1, confidence_2, 
                iou_1, iou_2, 
                frame_len, frame_stride, gif_duration,
                use_prior_data, use_pose, out_file, fps, 
                selected_ids, selected_labels, nearest_to_ball,
                x_padding, y_padding, x_interp, y_interp, zoom_interp],
        outputs=output
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860


* Running on public URL: https://19f2412d1c4c441a43.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
